In [37]:
# make sure these packages are installed
import pandas as pd
import numpy as np
import re
from datetime import datetime


# Python version of this notebook
from platform import python_version
print("python_version: "+ python_version())


SHEET_NAME_0 = 'Portfolios'
SHEET_NAME_1 = 'Sponsored Products Campaigns'
SHEET_NAME_2 = 'Sponsored Brands Campaigns'
SHEET_NAME_3 = 'Sponsored Display Campaigns'
entity_type_filter = {'Keyword', 'Product Targeting'}
ENABLED = 'enabled'
UPDATE = 'Update'


input_excel_file_name = ""
bid_multiplier_factor = 1.0
targeted_portfolio_names = set()

user_defined_parameters_file = open('user_defined_parameters.txt', 'r')
Lines = user_defined_parameters_file.readlines()

# current line number
count = 0

for line in Lines: 
    # extract input_excel_file_name on line 1
    if count == 1:
        input_excel_file_name = line[line.find("[")+1:line.find("]")].strip()
    # extract bid_multiplier_factor on line 3
    elif count == 3:
        bid_multiplier_factor = float(line[line.find("[")+1:line.find("]")])
    # extract targeted_portfolio_names from line 6 to the last line
    elif count >= 6:
        portfolio_name = line.strip()
        if len(portfolio_name) > 0:
            targeted_portfolio_names.add(portfolio_name)
            
    count += 1

print()
print("input_excel_file_name: " + input_excel_file_name)
print("bid_multiplier_factor: " + str(bid_multiplier_factor))
print("targeted_portfolio_names: ")
print(targeted_portfolio_names)
print()
    

df_portfolios = pd.read_excel(input_excel_file_name, SHEET_NAME_0)


# check duplicates in portfolio names.
if not df_portfolios['Portfolio Name'].is_unique:
    duplicate_portfolio_names = set()
    portfolio_names = set()
    
    for name in df_portfolios['Portfolio Name']:
        if name in portfolio_names:
            duplicate_portfolio_names.add(name)
        portfolio_names.add(name)
        
    raise Exception("Duplicates are found in the following portfolio names: " 
                        + '; '.join(duplicate_portfolio_names)
                       + '\nPlease eliminate duplicates before proceeding again. ')

print("No duplicates are found in portfolio names. You may proceed!")


def process_spreadsheet(file_name, sheet_name):
    campaign_df = pd.read_excel(file_name, sheet_name)
    
    campaign_df['Operation'] = campaign_df['Operation'].astype(str)
    
    campaign_df_filtered = campaign_df.loc[(campaign_df['Portfolio Name (Informational only)'].isin(targeted_portfolio_names))
                                         & (campaign_df['Entity'].isin(entity_type_filter))
                                         & (campaign_df['State'] == ENABLED)
                                         & (campaign_df['Campaign State (Informational only)'] == ENABLED)
                                         & (campaign_df['Ad Group State (Informational only)'] == ENABLED)]
    if campaign_df_filtered['Bid'].isnull().sum() > 0:
        raise Exception("Empty bids are found in the following rows: " 
                        + '; '.join(str(x) for x in campaign_df_filtered[campaign_df_filtered['Bid'].isnull()].index.tolist())
                       + '\nPlease eliminate empty bids before proceeding again. ')
    
    print("No empty bids are found. You may proceed!")
    
    campaign_df['Operation'] = np.where(
        ((campaign_df['Portfolio Name (Informational only)'].isin(targeted_portfolio_names)) 
       & (campaign_df['Entity'].isin(entity_type_filter))
       & (campaign_df['State'] == ENABLED)
       & (campaign_df['Campaign State (Informational only)'] == ENABLED)
       & (campaign_df['Ad Group State (Informational only)'] == ENABLED)    
        ),
        UPDATE,
        '')
    
    campaign_df['Bid'] = np.where(
        (campaign_df['Operation'] == UPDATE),
        round(campaign_df['Bid'] * bid_multiplier_factor + 0.00001, 2),
        campaign_df['Bid'])
    
    current_dateTime = datetime.utcnow().__str__().split('.')[0].replace(' ', '_').replace(':', '#')
    output_file_name = sheet_name.replace(' ', '_') + "_" + current_dateTime + "_" + file_name
    print("output_file_name: " + output_file_name)
    
    campaign_df.to_excel(output_file_name, 
                    sheet_name=sheet_name, 
                    index=False)  
    
    print('COMPLETED!')


process_spreadsheet(input_excel_file_name, SHEET_NAME_1)

python_version: 3.8.8

input_excel_file_name: no_empty_bids_test.xlsx
bid_multiplier_factor: 0.95
targeted_portfolio_names: 
{'csada', 'csdsa', '03013 OxfordMoving ACOS30', 'asd', 'sdas', '00A96 Rain Poncho ACOS25'}

No duplicates are found in portfolio names. You may proceed!
No empty bids are found. You may proceed!
output_file_name: Sponsored_Products_Campaigns_2023-03-11_05#54#15_no_empty_bids_test.xlsx
COMPLETED!
